In [1]:
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
# import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
# from resnets_utils import *
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

Using TensorFlow backend.



<img src="1.png" style="width:650px;height:350px;" >

![RESIDUAL](./RESID.png)

![RESIDUAL](./ex1.png)

![RESIDUAL](./idblock2_kiank.png)

In [2]:
def id_block(X , f ,filters):
    
    """
    input
    X 輸入的大小(維度)
    f 中間遮罩大小
    filters  list存遮罩數量
    stage 區域名稱
    block stage 下的 小名稱
    
    return 
    X   shape(n_H ,n_W , n_C)
    """
    
   
    
    #抓出F1 ,F2 ,F3
    F1 ,F2 ,F3=filters
    
    #記下CNN 的頭
    X_shortcut = X
    
    #CNN 原路
    #1
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid',
               kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    
    #2
    
    X = Conv2D(filters=F2, kernel_size=(f,f) , strides=(1,1) ,padding="same" ,
               kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3 )(X)
    X = Activation("relu")(X)
    
    #3
    
    X = Conv2D(filters=F3 , kernel_size=(1,1) ,strides=(1,1) ,padding="valid" ,
               kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3 )(X)
    
    #最後 主路線 加上捷徑
    
    X = Add()([X_shortcut , X])
    X = Activation("relu")(X)
    
    
    return X
    
    

![RESIDUAL](./2.png)

In [3]:
def convolutional_block(X, f, filters, s = 2):
    
    
    F1, F2, F3 = filters
        
    X_shortcut = X  
    # CNN 主路線
    #1
    X = Conv2D(F1, (1, 1), strides = (s,s),  kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    
    #2
    X = Conv2D(F2 , kernel_size=(f,f) ,strides=(1,1) ,padding="same" ,
               kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3 )(X)
    X = Activation("relu")(X)
    #3
    X = Conv2D(F3 ,kernel_size=(1,1) ,strides=(1,1) ,padding="valid" ,
              kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3 )(X)
    
    
    #捷徑
    X_shortcut = Conv2D(filters=F3 ,kernel_size=(1,1),strides=(s,s),padding="valid",
                       kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3 )(X_shortcut)
    
    X = Add()([X , X_shortcut])
    X = Activation("relu")(X)
    
    return X

![RESIDUAL](./3.png)

In [4]:
def ResNet50(input_shape = (28, 28, 1), classes = 10):
    
    
    
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), kernel_initializer = glorot_uniform(seed=0))(X) #綠
    X = BatchNormalization(axis = 3)(X)                                                   #橘
    X = Activation('relu')(X)                                                                                #黃
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)                                                              #紫

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256],  s = 1) #藍色
    X = id_block(X, 3, [64, 64, 256])                                   #紅色
    X = id_block(X, 3, [64, 64, 256])                                   #紅色

    

    # Stage 3
    X = convolutional_block(X,f=3,filters=[128,128,512],s=2)           #藍色
    X = id_block(X, f=3, filters=[128,128,512])                       #紅色
    X = id_block(X, f=3, filters=[128,128,512])                       #紅色
    X = id_block(X, f=3, filters=[128,128,512])                       #紅色
 
    # Stage 4
    X = convolutional_block(X, f=3, filters=[256,256,1024], s=2)        #藍色
    X = id_block(X, f=3, filters=[256,256,1024])                        #紅色
    X = id_block(X, f=3, filters=[256,256,1024])                        #紅色
    X = id_block(X, f=3, filters=[256,256,1024])                        #紅色
    X = id_block(X, f=3, filters=[256,256,1024])                        #紅色
    X = id_block(X, f=3, filters=[256,256,1024])                        #紅色

    # Stage 5 
    X = convolutional_block(X, f=3, filters=[512,512,2048], s=2)        #藍色
    X = id_block(X, f=3, filters=[512,512,2048])                        #紅色
    X = id_block(X, f=3, filters=[512,512,2048])                        #紅色
 
   
    X = AveragePooling2D(pool_size=(2,2),padding="same")(X)

    
    

    
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [5]:
from keras.datasets import mnist
from keras.utils import np_utils

In [6]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(60000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)
x_train = x_train/255
x_test = x_test/255

In [7]:
model = ResNet50(input_shape = (28, 28, 1), classes = 10)

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 34, 34, 1)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 14, 14, 64)   3200        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 14, 14, 64)   256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation

In [10]:
model.fit(x_train, y_train, epochs = 15, batch_size = 128)

Instructions for updating:
Use tf.cast instead.
Epoch 1/15
60000/60000 [==============================] - 53s 883us/step - loss: 0.2762 - acc: 0.9240
Epoch 2/15
60000/60000 [==============================] - 39s 654us/step - loss: 0.0630 - acc: 0.9812
Epoch 3/15
60000/60000 [==============================] - 39s 654us/step - loss: 0.0521 - acc: 0.9847
Epoch 4/15
60000/60000 [==============================] - 39s 654us/step - loss: 0.0435 - acc: 0.9868
Epoch 5/15
60000/60000 [==============================] - 39s 654us/step - loss: 0.0308 - acc: 0.9911
Epoch 6/15
60000/60000 [==============================] - 39s 654us/step - loss: 0.0261 - acc: 0.9922
Epoch 7/15
60000/60000 [==============================] - 39s 655us/step - loss: 0.0276 - acc: 0.9920
Epoch 8/15
60000/60000 [==============================] - 39s 654us/step - loss: 0.0295 - acc: 0.9910
Epoch 9/15
60000/60000 [==============================] - 39s 654us/step - loss: 0.0274 - acc: 0.9917
Epoch 10/15
60000/60000 [=========